<a href="https://colab.research.google.com/github/PavanGavit/FL_Lab/blob/main/FL__A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile server.py
import flwr as fl
import numpy as np


class LoggingFedAvg(fl.server.strategy.FedAvg):
    def aggregate_evaluate(self, server_round, results, failures):
        # Let Flower aggregate loss as usual
        aggregated_loss, _ = super().aggregate_evaluate(
            server_round, results, failures
        )

        # Manually aggregate accuracy (weighted by number of examples)
        total_examples = 0
        weighted_acc_sum = 0.0

        for _, eval_res in results:
            metrics = eval_res.metrics
            if metrics and "accuracy" in metrics:
                num_examples = eval_res.num_examples
                weighted_acc_sum += metrics["accuracy"] * num_examples
                total_examples += num_examples

        if total_examples > 0:
            avg_accuracy = weighted_acc_sum / total_examples
            print(
                f"[SERVER] Round {server_round} accuracy: {avg_accuracy:.4f}"
            )

        return aggregated_loss, {}  # metrics already logged manually


strategy = LoggingFedAvg(
    min_fit_clients=2,
    min_evaluate_clients=2,
    min_available_clients=2,
)

fl.server.start_server(
    server_address="0.0.0.0:8081",
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=strategy,
)


Overwriting server.py


In [ ]:
%%writefile client.py
import sys
import flwr as fl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

CLIENT_ID = int(sys.argv[1])
NUM_CLIENTS = 2

LOCAL_EPOCHS = {
    0: 1,
    1: 3,
}


def load_data(client_id: int, num_clients: int):
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    x_train = x_train / 255.0
    x_test = x_test / 255.0

    shard_size = len(x_train) // num_clients
    start = client_id * shard_size
    end = start + shard_size

    return (
        x_train[start:end],
        y_train[start:end],
        x_test,
        y_test,
    )


def build_model():
    model = keras.Sequential([
        keras.Input(shape=(28, 28)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ])

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model


class FlowerClient(fl.client.NumPyClient):
    def __init__(self, client_id: int):
        self.client_id = client_id
        self.model = build_model()
        self.x_train, self.y_train, self.x_test, self.y_test = load_data(
            client_id, NUM_CLIENTS
        )

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)

        epochs = LOCAL_EPOCHS[self.client_id]

        self.model.fit(
            self.x_train,
            self.y_train,
            epochs=epochs,
            batch_size=32,
            verbose=0,
        )

        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)

        loss, accuracy = self.model.evaluate(
            self.x_test,
            self.y_test,
            verbose=0,
        )

        return loss, len(self.x_test), {"accuracy": accuracy}


client = FlowerClient(CLIENT_ID).to_client()

fl.client.start_client(
    server_address="127.0.0.1:8081",
    client=client,
)


Overwriting client.py


In [ ]:
import subprocess
import time
import threading
import sys


def stream_output(pipe, prefix=""):
    for line in iter(pipe.readline, b""):
        print(prefix + line.decode(), end="")


# Start server with stdout captured
server = subprocess.Popen(
    ["python", "server.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)

# Stream server logs
server_thread = threading.Thread(
    target=stream_output,
    args=(server.stdout, "[SERVER] "),
    daemon=True,
)
server_thread.start()

time.sleep(3)

# Start client 0
client0 = subprocess.Popen(
    ["python", "client.py", "0"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)

client0_thread = threading.Thread(
    target=stream_output,
    args=(client0.stdout, "[CLIENT 0] "),
    daemon=True,
)
client0_thread.start()

time.sleep(1)

# Start client 1 (foreground)
client1 = subprocess.Popen(
    ["python", "client.py", "1"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)

stream_output(client1.stdout, "[CLIENT 1] ")

# Cleanup
client0.terminate()
server.terminate()

print("\n✅ Training Complete")


[SERVER] WARNING :   DEPRECATED FEATURE: flwr.server.start_server() is deprecated.
[SERVER] 	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:
[SERVER] 
[SERVER] 		$ flower-superlink --insecure
[SERVER] 
[SERVER] 	To view usage and all available options, run:
[SERVER] 
[SERVER] 		$ flower-superlink --help
[SERVER] 
[SERVER] 	Using `start_server()` is deprecated.
[SERVER] 
[SERVER]             This is a deprecated feature. It will be removed
[SERVER]             entirely in future versions of Flower.
[SERVER]         
[SERVER] INFO :      Starting Flower server, config: num_rounds=3, no round_timeout
[SERVER] INFO :      Flower ECE: gRPC server running (3 rounds), SSL is disabled
[SERVER] INFO :      [INIT]
[SERVER] INFO :      Requesting initial parameters from one random client
[CLIENT 0] 2026-01-21 09:30:51.910769: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
[CLIENT 0] 2026-